In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_part_number = df['Part_Number'].tolist()
list_brand = df['Brand'].tolist()
list_url = ['https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=' + part_number + '&brand=' + list_brand for part_number, list_brand in zip(list_part_number, list_brand)]

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests
import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for part_number, url in zip(list_part_number, list_url):
    work.put_nowait((part_number, url))

def crawler():
    global df
    
    while not work.empty():
        part_number, url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = json.loads(requests.get(url,
                                               headers=UA.get_User_Agent_Requests(),
                                               proxies=Proxy.get_Proxy_Requests()).text)
                
                # = = = = = = = = = = = = = = =
                
                if resp == []:
                    df_temp = pd.DataFrame([{'status': 'ok',
                                             'Part_Number': part_number,
                                             'Src': ''}])
                else:
                    df_temp = pd.DataFrame([{'status': 'ok',
                                             'Part_Number': part_number,
                                             'Src': resp[0]['image_BG_Url'].strip()}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Part_Number': part_number}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part_number +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./src_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：444

ADR103  <->  [ok] - 剩余数量：439
ADR102  <->  [ok] - 剩余数量：438
ADR106  <->  [ok] - 剩余数量：437
ADR105  <->  [ok] - 剩余数量：436
ADR101  <->  [ok] - 剩余数量：435
ADR108  <->  [ok] - 剩余数量：434
ADR112  <->  [ok] - 剩余数量：433
ADR107  <->  [ok] - 剩余数量：432
ADR115  <->  [ok] - 剩余数量：431
ADR111  <->  [ok] - 剩余数量：430
ADR120  <->  [ok] - 剩余数量：429
ADR116  <->  [ok] - 剩余数量：428
ADR121  <->  [ok] - 剩余数量：427
ADR125  <->  [ok] - 剩余数量：426
ADR122  <->  [ok] - 剩余数量：425
ADR127  <->  [ok] - 剩余数量：424
ADR126  <->  [ok] - 剩余数量：423
ADR129  <->  [ok] - 剩余数量：422
ADR128  <->  [ok] - 剩余数量：421
ADR130  <->  [ok] - 剩余数量：420
ADR124  <->  [ok] - 剩余数量：419
ADR133  <->  [ok] - 剩余数量：418
ADR134  <->  [ok] - 剩余数量：417
ADR135  <->  [ok] - 剩余数量：416
ADR132  <->  [ok] - 剩余数量：415
ADR139  <->  [ok] - 剩余数量：414
ADR136  <->  [ok] - 剩余数量：413
ADR141  <->  [ok] - 剩余数量：412
ADR138  <->  [ok] - 剩余数量：411
ADR143  <->  [ok] - 剩余数量：410
ADR137  <->  [ok] - 剩余数量：409
ADR144  <->  [ok] - 剩余数量：408
ADR142  <->  [ok] - 剩余数量：407
ADR148  <->  [ok] - 剩余数量：406
ADR15

ADR4418  <->  [ok] - 剩余数量：162
ADR4420  <->  [ok] - 剩余数量：161
ADR4422  <->  [ok] - 剩余数量：160
ADR4413  <->  [ok] - 剩余数量：159
ADR4423  <->  [ok] - 剩余数量：158
ADR4426  <->  [ok] - 剩余数量：157
ADR4428  <->  [ok] - 剩余数量：156
ADR4419  <->  [ok] - 剩余数量：155
ADR4433  <->  [ok] - 剩余数量：154
ADR4436  <->  [ok] - 剩余数量：153
ADR4429  <->  [ok] - 剩余数量：152
ADR4439  <->  [ok] - 剩余数量：151
ADR4440  <->  [ok] - 剩余数量：150
ADR4434  <->  [ok] - 剩余数量：149
ADR4443  <->  [ok] - 剩余数量：148
ADR4445  <->  [ok] - 剩余数量：147
ADR4444  <->  [ok] - 剩余数量：146
ADR4415  <->  [ok] - 剩余数量：145
ADR4446  <->  [ok] - 剩余数量：144
ADR4442  <->  [ok] - 剩余数量：143
ADR4451  <->  [ok] - 剩余数量：142
ADR4449  <->  [ok] - 剩余数量：141
ADR4455  <->  [ok] - 剩余数量：140
ADR4447  <->  [ok] - 剩余数量：139
ADR4456  <->  [ok] - 剩余数量：138
ADR4448  <->  [ok] - 剩余数量：137
ADR4454  <->  [ok] - 剩余数量：136
ADR4462  <->  [ok] - 剩余数量：135
ADR4460  <->  [ok] - 剩余数量：134
ADR4466  <->  [ok] - 剩余数量：133
ADR4470  <->  [ok] - 剩余数量：132
ADR4458  <->  [ok] - 剩余数量：131
ADR4474  <->  [ok] - 剩余数量：130
ADR4476  <